In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent)) 
from tool_monkey import MonkeyObserver, with_monkey, expired_token, forbidden_access, logger, create_tool_with_monkey, setup_default_logging, AuthenticationError

In [2]:
setup_default_logging(level=10)

Example 1: Using `bind_tools()` and `expired_token()`

In [ ]:
def bind_tools_expired_token_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_search_spotify_tracks, base_get_user_playlists, UserPlaylistInput
    from langchain_core.tools import tool 
    
    observer = MonkeyObserver()
    scenario = expired_token(on_call=1)
    
    # Search doesn't require auth, so no chaos
    @tool
    def search_spotify(query: str, limit: int = 5):
        """
        Search for tracks on Spotify.
        Args:
            query (str): Search query for songs, artists, or albums.
            limit (int): Maximum number of results to return.
        """
        return base_search_spotify_tracks(query=query, limit=limit)

    
    get_playlists=create_tool_with_monkey(
        base_tool=base_get_user_playlists,
        scenario=scenario,
        observer=observer,
        tool_name="get_playlists", 
        args_schema=UserPlaylistInput
    )
    system_prompt = """You are a helpful music assistant with access to Spotify tools.
When asked about playlists, use the get_playlists tool to fetch the user's playlists."""

    user_prompt = """Show me my Spotify playlists and tell me about them."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    llm_with_tools = llm.bind_tools([search_spotify, get_playlists])
    max_iterations = 10
    
    for i in range(max_iterations):
        ai_msg = llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        
        if not ai_msg.tool_calls:
            print(f"Agent loop finished: {ai_msg.content}")
            break
        
        for tool_call in ai_msg.tool_calls:
            tool_name = tool_call["name"]
            try:
                if tool_name == "search_spotify":
                    result = search_spotify.invoke(tool_call)
                    messages.append(result)
                    print(f"✅ Searched Spotify successfully")
                elif tool_name == "get_playlists":
                    result = get_playlists.invoke(tool_call)
                    messages.append(result)
                    print(f"✅ Retrieved playlists successfully")
            except AuthenticationError as e:
                print(f"❌ Authentication failed: {e}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": f"Error: {e}"
                })
    
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

bind_tools_expired_token_example()

Ending call for base_get_user_playlists after exception
❌ Authentication failed: ʕ•͡-•ʔ Tool Monkey unleashed! ʕ•͡-•ʔ: Access token expired
Agent loop finished: I'm sorry, but it seems like your Spotify access token has expired. Please renew it so I can fetch your playlists for you.

OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 1
  Success Rate: 0.0%
  Failures: 1
  Total Retries: 0
  Avg Latency: 0.3ms


Example 2: Using `bind_tools()` and `forbidden_access()`

In [4]:
def bind_tools_forbidden_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_add_tracks_to_playlist
    from langchain_core.tools import tool 
    
    observer = MonkeyObserver()
    # User doesn't have permission to modify playlists
    scenario = forbidden_access()
    
    # Add tracks requires write permission - wrap with chaos
    wrapped_add_tool = with_monkey(scenario, observer)(base_add_tracks_to_playlist)
    
    @tool
    def add_tracks(playlist_id: str, track_ids: list):
        """
        Add tracks to a Spotify playlist. Requires write permissions.
        Args:
            playlist_id (str): ID of the playlist to add tracks to.
            track_ids (list): List of track IDs to add.
        """
        return wrapped_add_tool(playlist_id=playlist_id, track_ids=track_ids)
    
    system_prompt = """You are a helpful music assistant with access to Spotify tools.
When asked to add songs to playlists, use the add_tracks tool."""

    user_prompt = """Add the tracks track1, track2, and track3 to my playlist playlist1."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    llm_with_tools = llm.bind_tools([add_tracks])
    max_iterations = 5
    
    for i in range(max_iterations):
        ai_msg = llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        
        if not ai_msg.tool_calls:
            print(f"Agent loop finished: {ai_msg.content}")
            break
        
        for tool_call in ai_msg.tool_calls:
            try:
                result = add_tracks.invoke(tool_call)
                messages.append(result)
                print(f"✅ Added tracks successfully")
            except AuthenticationError as e:
                print(f"❌ Permission denied: {e}")
                print(f"   Status code: {e.status_code}")
                print(f"   Failure type: {e.failure_type}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": f"Error: {e}"
                })
    
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

bind_tools_forbidden_example()

Ending call for base_add_tracks_to_playlist after exception
❌ Permission denied: ʕ•͡-•ʔ Tool Monkey unleashed! ʕ•͡-•ʔ: Insufficient permissions
   Status code: 403
   Failure type: forbidden
Agent loop finished: I'm sorry, but it seems I don't have the necessary permissions to add tracks to your playlist. Please make sure I have the correct permissions and try again.

OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 1
  Success Rate: 0.0%
  Failures: 1
  Total Retries: 0
  Avg Latency: 0.2ms
